In [51]:
   
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
#from tensorflow.python.framework import ops from cnn_utils import *

from math import pi
import itertools
import time

%matplotlib inline

with tf.Session() as sess:
    hello = tf.constant('Hello, TensorFlow!')
    print(sess.run(hello))


b'Hello, TensorFlow!'


In [69]:
class Vertex:
    """a polar coordinate vertex of unit length"""
    def __init__(self, index):
        self.index = index
        self.theta = tf.get_variable("theta" + str(index), [1], initializer=tf.constant_initializer(np.random.random_sample() * pi))
        self.phi = tf.get_variable("phi" + str(index), [1], initializer=tf.constant_initializer(np.random.random_sample() * 2 * pi))
        sin_theta = tf.sin(self.theta)
        self.x = tf.multiply(sin_theta, tf.cos(self.phi) , name='x' + str(self.index))
        self.y = tf.multiply(sin_theta, tf.sin(self.phi) , name='y' + str(self.index))
        self.z = tf.cos(self.theta, name='z' + str(self.index))
        self.xyz = tf.concat([self.x, self.y, self.z], axis=0)
    
    def distSquared( vertex1, vertex2 ):
        
        return 


def make_vertextes( n_vertexes ):
    print('make_vertextes')
    vertexes = []
    for i in range(n_vertexes):
        vertexes.append( Vertex(i) )
    return vertexes

def make_pairs( vertexes ):
    print('make Pairs Inverse Distance')
    list = []
    for pair in itertools.combinations(vertexes, 2):
        list.append( 1 / tf.norm(pair[0].xyz-pair[1].xyz))
    return tf.add_n(list)    



In [102]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
from sklearn import preprocessing

from pythreejs import *
import numpy as np
from IPython.display import display

def make_face_normals(pos,simplices):
    print(pos)
    print(simplices)
    face_normals = np.zeros((simplices.shape[0],3))
    for i in range(simplices.shape[0]):
        v = face_normals[i]
        for vertex_index in simplices[i]:
            #print(vertex_index)
            v += pos[vertex_index]
            
    face_normals = preprocessing.normalize(face_normals, norm='l2')   
    print(face_normals) 
    
    tuples=[]
    for i in range(face_normals.shape[0]):
        tuples.append((face_normals[i],i))
    return tuples    
 
def makeMeshes(pos):
    hull = ConvexHull(pos)
    
    # https://github.com/jovyan/pythreejs
    cubeGeometry = PlainGeometry(vertices=pos, faces=hull.simplices)   
    # , faceNormals=make_face_normals(pos,hull.simplices)
    
    myobjectCube = Mesh(geometry=cubeGeometry, material = LambertMaterial(color='#aaaaaa', opacity=0.75))
    cCube = PerspectiveCamera(position=[3, 3, 3], fov=25, aspect=1.0,
                          children=[DirectionalLight(color='#555555', position=[30, 30, 30], intensity=1.5)])
    sceneCube = Scene(children=[myobjectCube, AmbientLight(color='#ffffff')])

    renderer = Renderer(camera=cCube, background='black', background_opacity=1,
                            scene = sceneCube, controls=[OrbitControls(controlling=cCube)], width='800',height='800')
   
    return renderer

In [105]:
n_vertexes = 13

start_time = time.time()

tf.reset_default_graph()
verts = make_vertextes(n_vertexes)
potential = make_pairs(verts)
learning_rate = 1e-2 
multiplier = 1.05
learning_rate_placeholder = tf.placeholder(tf.float32, [], name='learning_rate')
print('make optimizer') 
optimizer = tf.train.GradientDescentOptimizer( learning_rate=learning_rate_placeholder ).minimize(potential)

#print([v.xyz for v in verts])
allPositions = tf.stack([v.xyz for v in verts])

NUM_THREADS = 2
 
print('start session') 
with tf.Session(config=tf.ConfigProto( intra_op_parallelism_threads=NUM_THREADS)) as sess:
    sess.run(tf.global_variables_initializer())
    last_p = 0
    for i in range(750):
        if(i % 1 == 0):
            _ , p = sess.run([optimizer, potential], feed_dict={learning_rate_placeholder: learning_rate} )
            if(i % 10 == 0):
                print( str(i) + ' ' + str(p) + ' learning_rate = ' + 
                      str( learning_rate) + ' multiplier = ' + str(multiplier) )
            if( abs(p - last_p) < 1e-6):
                print('early stop')
                break
            if( p < last_p ):
                learning_rate *= multiplier
            else:
                learning_rate /= 4
                multiplier = 1 + (multiplier-1)/2
            last_p = p;
        else:
            sess.run( optimizer, feed_dict={learning_rate_placeholder: learning_rate} )  
    
    elapsed_time = time.time() - start_time
    print( str(elapsed_time / 1e6)  + ' sec')
    
    
    pos = sess.run(allPositions, feed_dict={learning_rate_placeholder: learning_rate})    
    #print(pos)
    
    

make_vertextes
make Pairs Inverse Distance
make optimizer
start session
0 71.5051 learning_rate = 0.01 multiplier = 1.05
10 63.7297 learning_rate = 0.003122157424869163 multiplier = 1.025
20 62.4422 learning_rate = 0.003996625464122913 multiplier = 1.025
30 61.5253 learning_rate = 0.00511601848556533 multiplier = 1.025
40 60.8054 learning_rate = 0.00654893619119503 multiplier = 1.025
50 60.2191 learning_rate = 0.008383191999276915 multiplier = 1.025
60 59.7633 learning_rate = 0.010731194509304931 multiplier = 1.025
70 59.4539 learning_rate = 0.013736836232126048 multiplier = 1.025
80 59.2777 learning_rate = 0.017584311746901067 multiplier = 1.025
90 59.1834 learning_rate = 0.022509405687538498 multiplier = 1.025
100 59.1165 learning_rate = 0.028813942319663596 multiplier = 1.025
110 59.046 learning_rate = 0.03688428222076669 multiplier = 1.025
120 58.9684 learning_rate = 0.0472149995945799 multiplier = 1.025
130 58.9047 learning_rate = 0.06043919123525898 multiplier = 1.025
140 58.872 

In [106]:


#myobjectCube.material.wireframe = True

display(makeMeshes(pos))

Renderer(background_opacity=1.0, camera=PerspectiveCamera(aspect=1.0, children=[DirectionalLight(color='#555555', intensity=1.5, position=[30.0, 30.0, 30.0], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0])], fov=25.0, position=[3.0, 3.0, 3.0], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), controls=[OrbitControls(controlling=PerspectiveCamera(aspect=1.0, children=[DirectionalLight(color='#555555', intensity=1.5, position=[30.0, 30.0, 30.0], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0])], fov=25.0, position=[3.0, 3.0, 3.0], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), target=[0.0, 0.0, 0.0])], effect=None, height='800', scene=Scene(children=[Mesh(geometry=PlainGeometry(faces=array([[ 7,  4,  8],
       [ 3,  4,  8],
       [12,  6,  9],
       [12,  6,  2],
       [11,  6,  2],
       [11,  3,  8],
       [11,  3,  2],
       [ 1,  7,  4],
       [ 1, 12,  9],
       [ 0,  6,  9],
       [ 0,  1,  9],
       [ 0,  1,  7],
       [10,  3,  2],
       [10, 12,  2],
       [10,  3,  4],
       [10,  1,  4],
       [10,  1, 12],
       [ 5, 11,  6],
       [ 5,  0,  6],
       [ 5, 11,  8],
       [ 5,  7,  8],
       [ 5,  0,  7]], dtype=uint32), vertices=array([[ 0.5053882 ,  0.23298261,  0.83084404],
       [ 0.75141025, -0.64609212,  0.13396843],
       [-0.21058516,  0.42062876, -0.88245416],
       [-0.7658456 , -0.33537287, -0.54863971],
       [-0.24479751, -0.96796864,  0.05577525],
       [-0.33871365,  0.60126299,  0.72370982],
       [ 0.19084619,  0.97828996, -0.08078659],
       [-0.0201169 , -0.51841116,  0.85489482],
       [-0.86743712, -0.21727516,  0.44759843],
       [ 0.93440074,  0.34763807,  0.07773698],
       [ 0.17584358, -0.64785588, -0.74118942],
       [-0.77847177,  0.61358052, -0.13228986],
       [ 0.65798157,  0.13607584, -0.7406373 ]], dtype=float32)), material=LambertMaterial(color='#aaaaaa', envMap=None, lightMap=None, map=None, opacity=0.75, specularMap=None), position=[0.0, 0.0, 0.0], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), AmbientLight(color='#ffffff', position=[0.0, 0.0, 0.0], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0])], position=[0.0, 0.0, 0.0], scale=[1.0, 1.0, 1.0], up=[0.0, 1.0, 0.0]), width='800')

Todo:
1. fix face normals, https://github.com/mrdoob/three.js/blob/dev/src/core/Geometry.js computeFaceNormals
cubeGeometry.faceNormals

2. edge stats bar chart
4. seperate setup from converging loop so can run multiple if doesn't converge


## 12 Vertexes
![exampleOutput.png](attachment:exampleOutput.png)


## 32 Vertexes
![output%20for%2032%20vertexes.png](attachment:output%20for%2032%20vertexes.png)


In [63]:
pos.__class__

numpy.ndarray

In [91]:
cubeGeometry.faceNormals.__class__


tuple